In [5]:
import logging
import os
from datetime import datetime
print(datetime.now())

2023-12-07 10:28:01.239649


In [6]:
LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [7]:
os.getcwd()

'c:\\Users\\DEELL\\Desktop\\fsdsmendtoend\\notebook'

In [8]:
os.path.join(os.getcwd(),"logs")

'c:\\Users\\DEELL\\Desktop\\fsdsmendtoend\\notebook\\logs'

In [1]:
from src.DimondPricePridiction.logger import logging

In [2]:
logging.info(' i have just tested the things')